## knock 067 除算結果に対して有効桁数未満を切り上げる

In [4]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [5]:
df_product = pl.read_csv("../docker/work/data/product.csv")
display(df_product.head())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
str,i64,i64,i64,i64,i64
"""P040101001""",4,401,40101,198,149
"""P040101002""",4,401,40101,218,164
"""P040101003""",4,401,40101,230,173
"""P040101004""",4,401,40101,248,186
"""P040101005""",4,401,40101,268,201


### ノック

In [6]:
(
    df_product
    # 使用する列を抽出する(大規模データの場合メモリ使用量を節約する効果がある)
    .select([
        pl.col("unit_price"),
        pl.col("unit_cost")
    ])
    # 利益率30%の販売価格を計算する
    .with_columns(
        (pl.col("unit_cost") / 0.7)
        .ceil()
        .alias("new_price")
    )
    # 新しい販売価格の利益率を計算する
    .with_columns(
        ((pl.col("new_price") - pl.col("unit_cost")) / pl.col("new_price"))
        .alias("new_profit_rate")
    )
    # 先頭10行
    .head(n = 10)
)

unit_price,unit_cost,new_price,new_profit_rate
i64,i64,f64,f64
198,149,213.0,0.300469
218,164,235.0,0.302128
230,173,248.0,0.302419
248,186,266.0,0.300752
268,201,288.0,0.302083
298,224,320.0,0.3
338,254,363.0,0.300275
420,315,451.0,0.301552
498,374,535.0,0.300935
